In [1]:
# more_slowly_bare_mininum_encompassing


In [2]:
import pandas as pd
import numpy as np
import random
from random import randrange
import time
import os
import math
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import style
from mpl_toolkits.mplot3d import Axes3D
import itertools
from itertools import permutations
from itertools import combinations
import copy

In [3]:
environment_list_index_key_redesign = {
    0: 'is_occupied', # 0 = False/unoccupied, 1 = True/occupied
    1: 'is_ground', # 0 = False/not on ground, 1 = True/on ground
    2: 'terrain', # 0 = normal, 1 = difficult, 2 = unpassable
    3: 'light_level', # 0 = bright, 1 = dim, 2 = dark, 3 = magical darkness
    4: 'obscurement', # 0 = none, 1 = light, 2 = heavy
    5: 'entity_occupying', # a list of entities in the space
    6: 'object_occupying', # a list of objects in the space
    7: 'fill_type', # 0 = air, 1 = water, 2 = fire, 3 = earth, 4 = plants, 5 = ice
    8: 'sound_level', # 0 = quiet, 1 = normal, 2 = loud
    9: 'odor', # 0 = none, 1 = light, 2 = heavy
    10: 'piousness', # 0 = normal, 1 = consecrated, 2 = desecrated
    11: 'illusion', # 0 = no, 1 = yes
    12: 'wind', # 0 = none, 1 = light, 2 = strong
    13: 'temperature', # 0 = normal, 1= extreme cold, 2 = extreme heat
    14: 'biome', # 0 = none, 1 = arctic, 2 = coastal, 3 = desert, 4 = forest, 5 = grassland, 6 = hill, 7 = mountain, 8 = swamp, 9 = underdark, 10 = underwater, 11 = urban
}


In [4]:
def chebyshev_distance(point1, point2):
    """
    Calculate the Chebyshev distance between two points.
    
    Parameters:
    point1 (tuple): The (x, y) coordinates of the first point
    point2 (tuple): The (x, y) coordinates of the second point
    
    Returns:
    int: The Chebyshev distance between the two points
    """
    return max(abs(point1[0] - point2[0]), abs(point1[1] - point2[1]))

def euclidean_distance(point1, point2):
    """Calculate the Euclidean distance between two points."""
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

def octile_distance(point1, point2):
    """Calculate the octile distance between two points."""
    dx = abs(point1[0] - point2[0])
    dy = abs(point1[1] - point2[1])
    return max(dx, dy) + (math.sqrt(2) - 1) * min(dx, dy)


In [5]:
class world:
    def __init__(self, length, width): #, height):
        self.length = length
        self.width = width
        #self.height = height
        self.grid = self.generate_world()


        self.queue = {
            'action':      [],
            'destination': [],
            'metadata':     [],
        }

    def generate_world(self):
        # instead of numpy, lets use a list of lists of lists
        grid = []
        self.defender_locations = []
        self.defender_hps = []

        for x in range(self.length):
            grid.append([])
            for y in range(self.width):
                grid[x].append([0,0,0,0,0,[],[],0,0,0,0,0,0,0,0])                    
                grid[x][y][1] = 1                          

        def create_start_point(self):
            x = randrange(self.length)
            y = randrange(self.width)
            grid[x][y][3] = 1                                    
            self.start_point = [x,y]

        def create_end_point(self):
            # end point cannot be the same as the start point
            x = randrange(self.length)
            y = randrange(self.width)                  
            
            if grid[x][y][0] == 1:                                 
                create_end_point(self)
            else:
                grid[x][y][3] = 2                                   
                self.end_point = [x,y]                                
            
        def spawn_entity(self):
            # at the start point
            x = self.start_point[0]
            y = self.start_point[1]
                                                         
            grid[x][y][0] = 1                                 
            grid[x][y][4] = 1                               
            grid[x][y][5].append(1)                                       

            self.entity_location = [x, y]
            self.entity_hp = 5                 

        def spawn_defender(self):
            # defenders are spawned randomly throughout the world
            # they are not allowed to spawn at the start or end points
            # they prevent the entity from moving into their space or adjacent spaces
            x = randrange(self.length)
            y = randrange(self.width)

            if grid[x][y][0] == 1 or grid[x][y][3] == 1 or grid[x][y][3] == 2:   
                spawn_defender(self)
            else:
                grid[x][y][0] = 1                 # set occupied = True
                grid[x][y][5].append(2)            # set entity_occupied = True

            self.defender_locations.append([x,y])
            self.defender_hps.append(1)

        create_start_point(self)
        create_end_point(self)
        spawn_entity(self)
        spawn_defender(self)
        spawn_defender(self)
        spawn_defender(self)

        return world
    
    def move_entity(self):
        if self.entity_location == self.end_point:
            print('Entity has reached the end point')
            return
        
        grid = self.grid

        # move the entity
        # get the current location
        x = self.entity_location[0]
        y = self.entity_location[1]

        # get the possible moves
        possible_moves = []
        for x_offset in range(-1,2):
            for y_offset in range(-1,2):
                if x + x_offset >= 0 and x + x_offset < self.length and y + y_offset >= 0 and y + y_offset < self.width: 
                    if x_offset != 0 or y_offset != 0:
                        possible_moves.append([x + x_offset, y + y_offset])

        # remove the current location from the possible moves
        possible_moves = [move for move in possible_moves if move != [x,y]] 
        possible_move_scores = [0] * len(possible_moves)

        # calculate the score for each possible move
        for move in possible_moves:
            if grid[move[0]][move[1]][4] == 1:                 # if the path has been taken before
                possible_move_scores[possible_moves.index(move)] -= 10
            
            # need to calculate the distance from the end point
            distance_to_end_goal = chebyshev_distance(self.end_point, move)
            possible_move_scores[possible_moves.index(move)] -= distance_to_end_goal * 4

            # add bonus to moves that result in the end goal
            if move == self.end_point:
                possible_move_scores[possible_moves.index(move)] += 100

            defender_distances = []
            for defender in self.defender_locations:
                distance_from_defender = chebyshev_distance(defender,move)
                defender_distances.append(distance_from_defender)
            
            distance_from_start = chebyshev_distance(self.start_point, move)
            possible_move_scores[possible_moves.index(move)] += distance_from_start

            if min(defender_distances) == 1:
                possible_move_scores[possible_moves.index(move)] -= 10
            elif min(defender_distances) == 2:
                possible_move_scores[possible_moves.index(move)] -= 0
            elif min(defender_distances) == 3:
                possible_move_scores[possible_moves.index(move)] -= 0
            elif min(defender_distances) == 0:
                possible_move_scores[possible_moves.index(move)] -= 10
            
            for defender in self.defender_locations:
                if chebyshev_distance(defender, self.entity_location) ==1 and chebyshev_distance(defender, move) > 1:
                    possible_move_scores[possible_moves.index(move)] -= 10

        # sort the possible moves by score
        possible_moves, possible_move_scores = zip(*sorted(zip(possible_moves, possible_move_scores), key=lambda x: x[1], reverse=True))
        chosen_move = possible_moves[0]

        # move the entity
        self.grid[x][y][0] = 0 # set occupied = False
        self.grid[x][y][5].remove(1) # set entity_occupied = False

        self.grid[chosen_move[0]][chosen_move[1]][0] = 1 # set occupied = True
        self.grid[chosen_move[0]][chosen_move[1]][5].append(1) # set entity_occupied = True
        self.grid[chosen_move[0]][chosen_move[1]][4] = 1 # set obscurement = light

        # add info to queue
        self.queue['action'].append('move_entity')
        self.queue['destination'].append(None)
        self.queue['metadata'].append({
            'entity_location': self.entity_location,
            'chosen_move': chosen_move,
            'chosen_move_score': max(possible_move_scores),
            'possible_moves': possible_moves,
            'possible_move_scores': possible_move_scores,
        })

        self.entity_location = chosen_move
        if self.entity_location == self.end_point:
            print('Entity has reached the end point')
            return

        print(self.queue['metadata'][-1])

        return self.queue['metadata'][-1]

    def path_entity(self):
        pass

        

    def attack(self, attacker, destination):

        # check if within range?

        if destination == 'entity':
            self.entity_hp -= 1
    

        # pass to queue
        self.queue['action'].append('attack')
        self.queue['destination'].append(destination)
        self.queue['metadata'].append({
            'attacker': attacker,
            'destination': destination
        })


    def dash(self):
        pass

    def take_turn(self):
        pass
        

    def check_for_opportunity_attacks(self):
        for defender in self.defender_locations:
            # if the entity is 1 Space away from a defender, and the new move would place them more than 1 space away, the defender gets an opportunity attack
            if chebyshev_distance(defender, self.entity_location) ==1 and chebyshev_distance(defender, self.queue['metadata'][-1]['chosen_move']) > 1:
                self.queue['action'].append('opportunity_attack')
                self.queue['destination'].append('entity')
                self.queue['metadata'].append({
                    'attacker': defender,
                    'attacker location': defender.location
                    })
                
    def move_defender(self):
        for defender_pos in self.defender_locations:
            x = defender_pos[0]
            y = defender_pos[1]

            # get the possible moves
            possible_moves = []
            for x_offset in range(-1,2):
                for y_offset in range(-1,2):
                    if x + x_offset >= 0 and x + x_offset < self.length and y + y_offset >= 0 and y + y_offset < self.width: 
                        if x_offset != 0 or y_offset != 0:
                            possible_moves.append([x + x_offset, y + y_offset])
            
            # remove the current location from the possible moves
            possible_moves = [move for move in possible_moves if move != [x,y]]
            possible_move_scores = [0] * len(possible_moves)
            '''
            # calculate the score for each possible move
            for move in possible_moves:
                if self.grid[move[0]][move[1]][4] == 1: 
                    possible_move_scores[possible_moves.index(move)] -= 10
                
                # need to calculate the distance from the end point
                distance_to_end_goal = chebyshev_distance(self.end_point, move)
                possible_move_scores[possible_moves.index(move)] -= distance_to_end_goal * 4

                # add bonus to moves that result in the end goal
                if move == self.end_point:
                    possible_move_scores[possible_moves.index(move)] += 100

                entity_distance = chebyshev_distance(self.entity_location, move)
                possible_move_scores[possible_moves.index(move)] += entity_distance

                if entity_distance == 1:
                    possible_move_scores[possible_moves.index(move)] -= 10
                elif entity_distance == 2:
                    possible_move_scores[possible_moves.index(move)] -= 0
                elif entity_distance == 3:
                    possible_move_scores[possible_moves.index(move)] -= 0
                elif entity_distance == 0:
                    possible_move_scores[possible_moves.index(move)] -= 10
            '''
            # sort the possible moves by score
            possible_moves, possible_move_scores = zip(*sorted(zip(possible_moves, possible_move_scores), key=lambda x: x[1], reverse=True))
            chosen_move = possible_moves[0]

            # move the defender
            self.grid[x][y][0] = 0
            self.grid[x][y][5].remove(2)

            self.grid[chosen_move[0]][chosen_move[1]][0] = 1
            self.grid[chosen_move[0]][chosen_move[1]][5].append(2)

            self.defender_locations[self.defender_locations.index(defender_pos)] = chosen_move

            # add info to queue
            self.queue['action'].append('move_defender')
            self.queue['destination'].append(None)
            self.queue['metadata'].append({
                'defender_location': defender_pos,
                'chosen_move': chosen_move,
                'chosen_move_score': max(possible_move_scores),
                'possible_moves': possible_moves,
                'possible_move_scores': possible_move_scores,
            })



            
        

In [6]:
def visualize_3d_voxels(data):
    """
    Visualize a 3D list of lists of lists of lists using matplotlib's 3D voxels.
    The corner closest to the camera is set to (0,0,0).
    The first value in each innermost list determines the fill value.
    
    Parameters:
    data (list): A 3D list of lists of lists of lists to be visualized.
    
    Returns:
    None (displays the plot)
    """
    # Check if the input is a list of lists of lists of lists
    if not isinstance(data, list) or not all(isinstance(layer, list) for layer in data) or \
       not all(isinstance(row, list) for layer in data for row in layer) or \
       not all(isinstance(cell, list) for layer in data for row in layer for cell in row):
        raise ValueError("Input must be a 4D list (list of lists of lists of lists)")
    
    # Get the dimensions of the data
    x_dim, y_dim, z_dim = len(data), len(data[0]), len(data[0][0])
    
    # Get the fill value (first value in the first list)
    occupied_space = data[0][0][0][0]

    
    # Create a boolean array for plotting
    entity_array = np.array([[[sum(cell[5]) == 1 for cell in row] for row in layer] for layer in data]) # using occupied to represent entities
    dim_light_array = np.array([[[cell[3] == 1 for cell in row] for row in layer] for layer in data])             # using dim light to represent the start point
    dark_light_array = np.array([[[cell[3] == 2 for cell in row] for row in layer] for layer in data])            # using dark light to represent the end point
    path_array = np.array([[[cell[4] == 1 for cell in row] for row in layer] for layer in data])                  # using obscurement to represent spaces traversed
    
    defender_array = np.array([[[sum(cell[5]) == 2 for cell in row] for row in layer] for layer in data])               # using entity_occupying to represent defenders


    # Create a figure and a 3D axis
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    # Create coordinate arrays (add one to each dimension)
    x, y, z = np.meshgrid(
        np.arange(x_dim + 1),
        np.arange(y_dim + 1),
        np.arange(z_dim + 1),
        indexing='ij'
    )
    
    # Plot the voxels
    ax.voxels(x, y, z, entity_array, facecolors='cyan', edgecolor='k')
    ax.voxels(x, y, z, dim_light_array, facecolors='green', alpha=0.25)
    ax.voxels(x, y, z, dark_light_array, facecolors='white', alpha=0.5)
    ax.voxels(x, y, z, path_array, facecolors='orange', alpha=0.1)
    ax.voxels(x, y, z, defender_array, facecolors='red', edgecolor='k', alpha=0.5)
    
    # Set labels for each axis
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    
    
    # Flip the X-axis
    ax.invert_xaxis()
    
    # Set axis limits
    ax.set_xlim(x_dim, 0)
    ax.set_ylim(0, y_dim)
    ax.set_zlim(0, z_dim)
    
    # Add axis labels
    ax.set_xticks(np.arange(0, x_dim + 1, 1))
    ax.set_yticks(np.arange(0, y_dim + 1, 1))
    ax.set_zticks(np.arange(0, z_dim + 1, 1))
    
    # Show the plot
    plt.show()


In [7]:
def visualize_2d_voxels(data):
    """
    Visualize a 2D list of lists of lists using matplotlib's 3D voxels.
    The corner closest to the camera is set to (0,0,0).
    The first value in each innermost list determines the fill value.
    
    Parameters:
    data (list): A 2D list of lists of lists to be visualized.
    
    Returns:
    None (displays the plot)
    """
    # Check if the input is a list of lists of lists
    if not isinstance(data, list) or not all(isinstance(row, list) for row in data) or \
       not all(isinstance(cell, list) for row in data for cell in row):
        raise ValueError("Input must be a 3D list (list of lists of lists)")
    
    # Get the dimensions of the data
    x_dim, y_dim = len(data), len(data[0])
    z_dim = max(x_dim, y_dim) / 2   # We're working with 2D data, so z_dim is always 1
    
    # Get the fill value (first value in the first list)
    occupied_space = data[0][0][0]

    # Create boolean arrays for plotting
    entity_array = np.array([[sum(cell[5]) == 1 for cell in row] for row in data])
    dim_light_array = np.array([[cell[3] == 1 for cell in row] for row in data])
    dark_light_array = np.array([[cell[3] == 2 for cell in row] for row in data])
    entity_path_array = np.array([[cell[4] == 1 for cell in row] for row in data])
    defender_array = np.array([[sum(cell[5]) == 2 for cell in row] for row in data])
    defender_path_array = np.array([[cell[4] == 2 for cell in row] for row in data])

    # Create a figure and a 3D axis
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    # Create coordinate arrays
    x, y, z = np.meshgrid(
        np.arange(x_dim + 1),
        np.arange(y_dim + 1),
        np.arange(2),  # We only need 2 z-values for a single layer
        indexing='ij'
    )
    
    # Plot the voxels
    ax.voxels(x, y, z, entity_array[:,:,np.newaxis], facecolors='cyan', edgecolor='k')
    ax.voxels(x, y, z, dim_light_array[:,:,np.newaxis], facecolors='green', alpha=0.25)
    ax.voxels(x, y, z, dark_light_array[:,:,np.newaxis], facecolors='white', alpha=0.5)
    ax.voxels(x, y, z, entity_path_array[:,:,np.newaxis], facecolors='orange', alpha=0.1)
    ax.voxels(x, y, z, defender_array[:,:,np.newaxis], facecolors='red', edgecolor='k', alpha=0.5)
    ax.voxels(x, y, z, defender_path_array[:,:,np.newaxis], facecolors='purple', alpha=0.1)
    
    # Set labels for each axis
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    
    # Flip the X-axis
    ax.invert_xaxis()
    
    # Set axis limits
    ax.set_xlim(x_dim, 0)
    ax.set_ylim(0, y_dim)
    ax.set_zlim(0, z_dim)  # Only show one layer in z-direction
    
    # Add axis labels
    ax.set_xticks(np.arange(0, x_dim + 1, 1))
    ax.set_yticks(np.arange(0, y_dim + 1, 1))
    ax.set_zticks([0, z_dim])
    
    #ax.view_init(elev=20, azim=45)


    # Show the plot
    plt.show()

In [8]:
first_test = world(10, 10)
#first_test.path_entity()

first_test.move_entity()
first_test.move_entity()
first_test.move_entity()
first_test.move_entity()
first_test.move_entity()

TypeError: type 'world' is not subscriptable

In [ ]:
visualize_2d_voxels(first_test.grid)

ValueError: Input must be a 3D list (list of lists of lists)